# Hénon--Heiles potential

## Import libraries

In [ ]:
import numpy as np

import scipy.integrate as integrate
import scipy.interpolate as interp
import scipy.optimize as optimize

import mpl_toolkits.mplot3d as threeD
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib.transforms import Bbox

import os
if not os.path.isdir("./figs/"):
    os.makedirs("./figs/")

## Functions

In [ ]:
def V(x,y):
    """
    Calculates the Hénon-Heiles potential.
    Arguments:
        x: number or array of x coordinates
        y: number or array of y coordinates
    Returns:
        The height of potential in every (x,y) point
    """
    
    V = (1/2) * (x**2 + y**2) + x**2 * y - (1/3) * y**3
    
    return V





def derivation(t,y):
    """
    Returns the derivatives of the coordinates x,y,px,py. Input for RK45 solver.
    Arguments:
        t: t
        y: array of coordinates in the form: (x,y,px,py)
    Returns:
        The derivatives of every coordinate: (px,py, px_dot, py_dot)
    """
    
    x, y, px, py = y
    pxdot = -x - 2 * x * y
    pydot = -y - x**2 + y**2
    
    return [px, py, pxdot, pydot]





def energy_func(y, E=0):
    """
    Calculates the energy of the system. Or if the input energy is not zero returns
    the difference between the calculated and the given energy for optimalization.
    Arguments:
        y: array of coordinates in the form: (x,y,px,py)
        E: positive number
    Returns:
        The calculated energy of the system
    """
    x, y, px, py = y
    
    T = (1/2) * (px**2 + py**2)
    V = (1/2) * (x**2 + y**2) + x**2 * y - (1/3) * y**3
    
    return T + V - E




def solve_RK(init, steps = 5000, check_energy=True):
    """
    Generates data using the RK45 method.
    Arguments:
        init: initial values in the form: (x,y,px,py)
        steps: number of steps to take
    Returns:
        Time steps in an array, coordinates in a 4 by steps dimension array, energy of the system in an array.
    """
    rk = integrate.RK45(derivation, t0=0, y0=init, t_bound=1e12, rtol=1e-6, atol=1e-14)

    trajectory = []
    t = []
    
    for i in range(steps):
        t.append(rk.t)
        trajectory.append(rk.y)
        rk.step()
        if rk.status != "running":
            break
    
    solution_rk = np.array(trajectory).T
    energy = energy_func(solution_rk)
    if check_energy == True:
        if (abs(energy).max() - abs(energy[0])) > 0.001:
            print("Energy diverges: E_init = %.3e, E_max = %.3e" %(abs(energy[0]).max(), abs(energy).max()))
            return 0
    
    return np.array(t), solution_rk, energy



def lyapunov_indexes(solution_rk):
    """
    Calculates the lyapunov coefficients for solution_rk's energy.
    Arguments:
        solution_rk: coordinates in a 4 by steps dimension array
    Returns:
        The x and y lyapunov coefficients.
    """
    lyapunov_x = np.log(abs(solution_rk[0] + 2 * solution_rk[0] * solution_rk[1]))
    lyapunov_y = np.log(abs(solution_rk[1] + solution_rk[0]**2 - solution_rk[1]**2))
    
    return np.array((lyapunov_x.sum() / len(lyapunov_x), lyapunov_y.sum() / len(lyapunov_y)))



def plot_trajectory_in_potential(energy, lim=[-1,1], save=False):
    """
    Plots the trajectories of two close points and some equipotential lines. Can save the plot.
    Arguments:
        energy: The energy of the solutions.
        lim: minimum and maximum points on x and y axis of the plot
        save=False: If it is a string the file will be saved to "./figs/the string.svg".
    Returns: Nothing. 
    """
    
    initial_values = calc_initial_values(energy)
    _, solution_rk, energy_rk = solve_RK(init=initial_values, check_energy=False)
    rand = np.random.uniform(low=1e-10, high=1e-9)
    initial_values[0] = initial_values[0] + rand
    _, solution_rk_rand, energy_rk_rand = solve_RK(init=initial_values, check_energy=False)

    

    x_in = np.linspace(lim[0], lim[1], 1000)
    y_in = np.linspace(lim[0], lim[1], 1000)
    X_in, Y_in = np.meshgrid(x_in, y_in)
    Z_in = V(X_in, Y_in)

    plt.rcParams.update({'font.size': 15})
    plt.figure(figsize=(10,10))
    
    plt.plot(solution_rk[0], solution_rk[1], linewidth=0.5)
    plt.plot(solution_rk_rand[0], solution_rk_rand[1], color="orange", linewidth=0.5)
    plt.plot([], [], linestyle=" ", label="Steps taken: %d" %solution_rk.shape[1])
    plt.contour(X_in,Y_in,Z_in, levels=10)

    plt.xlabel(r'$x$', fontsize=20)
    plt.ylabel(r'$y$', fontsize=20)
    plt.xlim(lim[0], lim[1])
    plt.ylim(lim[0], lim[1])
    plt.legend(fontsize=20)
    
    if save != False:
        plt.savefig("./figs/" + save + ".svg", transparent=True, bbox_inches='tight')
    
    
def phase_space(energy, lim=[-1,1], save=False):
    """
    Generates phase space diagrams for two close initial values.
    Arguments:
        energy: the initial energy of two close points.
    Returns: Nothing.
    """
    initial_values = calc_initial_values(energy)
    _, solution_rk, _ = solve_RK(init=initial_values, check_energy=False)
    rand = np.random.uniform(low=1e-10, high=1e-9)
    initial_values[0] = initial_values[0] + rand
    _, solution_2, _ = solve_RK(init=initial_values, check_energy=False)
    
    
    plt.rcParams.update({'font.size': 13})
    fig, ax = plt.subplots(2,2,figsize=(20,20))
    
    if save == False:
        fig.suptitle('Phase space diagrams of two close points with energy %0.5f' %energy, fontsize=30)
        fig.tight_layout(pad=4)



    ax[0][0].plot(solution_rk[0], solution_rk[1], linewidth=0.7)
    ax[0][0].plot(solution_2[0], solution_2[1], linewidth=0.7)

    ax[0][0].plot([], [], linestyle=" ", label="Steps taken: %d" %solution_rk.shape[1])
    ax[0][0].set_xlabel(r'$x$', fontsize=20)
    ax[0][0].set_ylabel(r'$y$', fontsize=20)
    ax[0][0].set_xlim(lim[0], lim[1])
    ax[0][0].set_ylim(lim[0], lim[1])
    ax[0][0].legend(fontsize=25)



    ax[0][1].plot(solution_rk[2], solution_rk[3], linewidth=0.7)
    ax[0][1].plot(solution_2[2], solution_2[3], linewidth=0.7)

    ax[0][1].plot([], [], linestyle=" ", label="Steps taken: %d" %solution_rk.shape[1])
    ax[0][1].set_xlabel(r'$p_x$', fontsize=20)
    ax[0][1].set_ylabel(r'$p_y$', fontsize=20)
    ax[0][1].set_xlim(lim[0], lim[1])
    ax[0][1].set_ylim(lim[0], lim[1])
    ax[0][1].legend(fontsize=25)



    ax[1][0].plot(solution_rk[1], solution_rk[3], linewidth=0.7)
    ax[1][0].plot(solution_2[1], solution_2[3], linewidth=0.7)

    ax[1][0].plot([], [], linestyle=" ", label="Steps taken: %d" %solution_rk.shape[1])
    ax[1][0].set_xlabel(r'$y$', fontsize=20)
    ax[1][0].set_ylabel(r'$p_y$', fontsize=20)
    ax[1][0].set_xlim(lim[0], lim[1])
    ax[1][0].set_ylim(lim[0], lim[1])
    ax[1][0].legend(fontsize=25)



    ax[1][1].plot(solution_rk[0], solution_rk[2], linewidth=0.7)
    ax[1][1].plot(solution_2[0], solution_2[2], linewidth=0.7)

    ax[1][1].plot([], [], linestyle=" ", label="Steps taken: %d" %solution_rk.shape[1])
    ax[1][1].set_xlabel(r'$x$', fontsize=20)
    ax[1][1].set_ylabel(r'$p_x$', fontsize=20)
    ax[1][1].set_xlim(lim[0], lim[1])
    ax[1][1].set_ylim(lim[0], lim[1])
    ax[1][1].legend(fontsize=25)

    if save == True:
        fig.savefig("./figs/%0.3f_1.svg" %energy, transparent=True, bbox_inches=Bbox([[1, 10], [10, 18]]))
        fig.savefig("./figs/%0.3f_2.svg" %energy, transparent=True, bbox_inches=Bbox([[10, 10], [19, 18]]))
        fig.savefig("./figs/%0.3f_3.svg" %energy, transparent=True, bbox_inches=Bbox([[1, 1], [10, 10]]))
        fig.savefig("./figs/%0.3f_4.svg" %energy, transparent=True, bbox_inches=Bbox([[10, 1], [19, 10]]))
    plt.show()
    
    
    
def calc_initial_values(energy_level):
    """
    Calculates a random starting point in phase space given a fixed energy level.
    Arguments:
        energy_level: The energy level of the random point
    Returns: An array of (x,y,px,py) with the given energy.
    """
    px_max = np.sqrt(2* energy_level)
    px = np.random.uniform(low=-px_max, high=px_max)

    py_max = np.sqrt(2* (energy_level - 1/2 * px**2))
    py = np.random.uniform(low=-py_max, high=py_max)

    x_max = np.sqrt(2* (energy_level - 1/2 * px**2 - 1/2 * py**2))
    x = np.random.uniform(low=-x_max, high=x_max)

    reduced_energy = energy_level - 1/2 * px**2 - 1/2 * py**2 - 1/2 * x**2
    def minimize(y, x, reduced_energy):
        return 0.5 * y**2 + x**2 * y - 1/3 * y**3 - reduced_energy
    
    y = optimize.fsolve(minimize, x0=0.1, args=(x,reduced_energy))[0]
    
    if abs(energy_level - energy_func((x,y,px,py))) > 0.01:
        print("False initial values")
        return 0
    
    return np.array((x,y,px,py))



def poincare_section(energy, degree, calc_passing_velocity=False):
    """
    Calculates the poincaré section y, py given a fixed energy level. Calculates the instantanious velocity at the beginning of every period. Samples "degree" different trajectories.
    Arguments:
        energy: The energy level of the poincaré section
        degree: Number of trajectories to simulate
        calc_passing_velocity: If True, the function calculates the passing velocity and the time.
    Returns: Array of the y and py values. If calc_passing_velocity=True it returns the pass time and the velocity.
    """
    data_y = np.array([])
    data_py = np.array([])
    data_px = np.array([])
    rerun = 0
    i = 0
    while i < degree:
        i = i + 1
        initial_values = calc_initial_values(energy)
        t_rk, solution_rk, energy_rk = solve_RK(init=initial_values, steps=20000, check_energy=False)
        if (abs(energy_rk).max() - energy) > 0.01:
            i = i - 1
            rerun = rerun +1
            if rerun > 10:
                break
            continue
        

        crossing = []
        for j in range(solution_rk.shape[1] - 1):
            if (((solution_rk[0][j]) > 0 and (solution_rk[0][j+1]) < 0) or ((solution_rk[0][j]) < 0 and (solution_rk[0][j+1]) > 0)) and solution_rk[2][j] > 0:
                crossing.append(j)

        x_func = interp.interp1d(t_rk, solution_rk[0], kind='linear')
        crossing_t = []
        for cross in crossing:
            crossing_t.append(optimize.fsolve(x_func, x0=t_rk[cross]))
        crossing_t = np.reshape(crossing_t, len(crossing))

        y_func = interp.interp1d(t_rk, solution_rk[1], kind='linear')
        py_func = interp.interp1d(t_rk, solution_rk[3], kind='linear')
        y_interp = y_func(crossing_t)
        py_interp = py_func(crossing_t)
    
        data_y = np.concatenate((data_y, y_interp))
        data_py = np.concatenate((data_py, py_interp))
        if calc_passing_velocity == True:
            px_func = interp.interp1d(t_rk, solution_rk[2], kind='linear')
            px_interp = px_func(crossing_t)
            data_px = np.concatenate((data_px, px_interp))


    if rerun > 0:
        print("Number of times new initial values were generated, because the energy divegres: %d." %rerun)
    
    if calc_passing_velocity == True:
        passing_velocity = np.sqrt(px_interp**2 * py_interp**2)
        return crossing_t, passing_velocity
    return data_y, data_py
                    

                    
def plot_poincare(energy, degree=30):
    """
    Plots the poincaré section y, py on a fixed energy level. Plots "degree" different trajectories. 
    Arguments:
        energy: The energy level of the poincaré section
        degree: Number of trajectories to plot
    Returns: Nothing.
    """    
    poincare_y, poincare_py = poincare_section(energy, degree)

    plt.scatter(poincare_y, poincare_py, s=1, color="k")

    plt.plot([], [], ' ', label="E = %0.5f" %energy)
    plt.xlabel(r'$y$', fontsize=20)
    plt.ylabel(r'$p_y$', fontsize=20)
    plt.legend(fontsize=20)

## Visualization of potential

In [ ]:
x = np.linspace(-20, 20, 1000)
y = np.linspace(-20, 20, 1000)
X, Y = np.meshgrid(x, y)
Z = V(X, Y)



plt.rcParams.update({'font.size': 12})
fig = plt.figure(figsize=(10,10))
fig.tight_layout(pad=3)

ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z)
#ax.set_title('3D potential', fontsize=30)
ax.set_xlabel(r'$x$', fontsize=20)
ax.set_ylabel(r'$y$', fontsize=20)
ax.set_zlabel(r'$z$', fontsize=20)
fig.savefig("./figs/unbounded.svg", transparent=True, bbox_inches='tight')



plt.rcParams.update({'font.size': 12})
fig = plt.figure(figsize=(10,10))
fig.tight_layout(pad=3)

ax = fig.add_subplot(111)
ax.contour(X,Y,Z, levels=20)
#ax.set_title('Equipotential lines', fontsize=30)
ax.set_xlabel(r'$x$', fontsize=20)
ax.set_ylabel(r'$y$', fontsize=20)
fig.savefig("./figs/unbounded lines.svg", transparent=True, bbox_inches='tight')




x = np.linspace(-1, 1, 10000)
y = np.linspace(-1, 1, 10000)




plt.rcParams.update({'font.size': 12})
fig = plt.figure(figsize=(10,10))
fig.tight_layout(pad=3)

ax = fig.add_subplot(111)
ax.plot(x, V(x, np.zeros(len(x))))
#ax.set_title('Shape of potential in the x scale', fontsize=30)
ax.set_xlabel(r'$x$', fontsize=20)
ax.set_ylabel(r'V$(x, y=0)$', fontsize=20)
fig.savefig("./figs/x scale.svg", transparent=True, bbox_inches='tight')




plt.rcParams.update({'font.size': 12})
fig = plt.figure(figsize=(10,10))
fig.tight_layout(pad=3)

ax = fig.add_subplot(111)
ax.plot(y, V(np.zeros(len(y)), y))
#ax.set_title('Shape of potential in the y scale', fontsize=30)
ax.set_xlabel(r'$y$', fontsize=20)
ax.set_ylabel(r'V$(x=0, y)$', fontsize=20)
fig.savefig("./figs/y scale.svg", transparent=True, bbox_inches='tight')

## Bounded orbit

In [ ]:
x = np.linspace(-0.2, 0.2, 1000)
y = np.linspace(-0.2, 0.2, 1000)
X, Y = np.meshgrid(x, y)
Z = V(X, Y)




plt.rcParams.update({'font.size': 12})
fig = plt.figure(figsize=(10,10))
fig.tight_layout(pad=3)

ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z)
#ax.set_title('3D potential of bounded orbit', fontsize=30)
ax.set_xlabel(r'$x$', fontsize=20)
ax.set_ylabel(r'$y$', fontsize=20)
ax.set_zlabel(r'$z$', fontsize=20)
fig.savefig("./figs/bounded.svg", transparent=True, bbox_inches='tight')




plt.rcParams.update({'font.size': 12})
fig = plt.figure(figsize=(10,10))
fig.tight_layout(pad=3)

ax = fig.add_subplot(111)
ax.contour(X,Y,Z, levels=15)
#ax.set_title('Equipotential lines of bounded orbit', fontsize=30)
ax.set_xlabel(r'$x$', fontsize=20)
ax.set_ylabel(r'$y$', fontsize=20)
fig.savefig("./figs/bounded lines.svg", transparent=True, bbox_inches='tight')

## Visualization of solution

In [ ]:
plot_trajectory_in_potential(1/12, save="orbit12")

plot_trajectory_in_potential(1/6, save="orbit6")

plot_trajectory_in_potential(1/4, save="orbit4")

## Phase space diagrams

In [ ]:
phase_space(1/12, save=True)
plt.show()
phase_space(1/6, save=True)
plt.show()
phase_space(1/4, lim=[-2,2], save=True)
plt.show()

## Poincaré section

In [ ]:
#plt.rcParams.update({'font.size': 20})
#fig = plt.figure(figsize=(10,10))
#fig.add_subplot(111)
#plot_poincare(1/12)
#fig.savefig("./figs/poincare 1slash12.svg", transparent=True, bbox_inches='tight')
print("First")

#plt.rcParams.update({'font.size': 20})
#fig = plt.figure(figsize=(10,10))
#fig.add_subplot(111)
#plot_poincare(1/10)
#fig.savefig("./figs/poincare 1slash10.svg", transparent=True, bbox_inches='tight')
#print("Second")

#plt.rcParams.update({'font.size': 20})
#fig = plt.figure(figsize=(10,10))
#fig.add_subplot(111)
#plot_poincare(1/8)
#fig.savefig("./figs/poincare 1slash8.svg", transparent=True, bbox_inches='tight')
#print("Third")

#plt.rcParams.update({'font.size': 20})
#fig = plt.figure(figsize=(10,10))
#fig.add_subplot(111)
#plot_poincare(1/6)
#fig.savefig("./figs/poincare 1slash6.svg", transparent=True, bbox_inches='tight')
#print("Fourth")
#plt.show()

## Lyapunov coefficients

### Lyapunov coefficients and number of steps

In [ ]:
time = np.logspace(1, 5, 30, dtype="int")
energy_1 = 1/12
energy_2 = 1/4

lyapunov_matrix_1 = []
lyapunov_matrix_2 = []
initial_values_1 = calc_initial_values(energy_1)
initial_values_2 = calc_initial_values(energy_2)
for i in time:
    _, solution_rk_1, _ = solve_RK(init=initial_values_1, steps=i, check_energy=False)
    _, solution_rk_2, _ = solve_RK(init=initial_values_2, steps=i, check_energy=False)
    lyapunov_matrix_1.append(lyapunov_indexes(solution_rk_1))
    lyapunov_matrix_2.append(lyapunov_indexes(solution_rk_2))
    
lyapunov_matrix_1 = np.array(lyapunov_matrix_1).T
lyapunov_matrix_2 = np.array(lyapunov_matrix_2).T

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.scatter(time, lyapunov_matrix_1[0], label="Energy: %0.5f" %energy_1)
ax.scatter(time, lyapunov_matrix_2[0], label="Energy: %0.5f" %energy_2)
ax.axhline(0, color="k", label="Separation of stable and chaotic behavior")

#ax.set_title("Lyapunov coefficient in x direction", fontsize=30)
ax.set_xscale("log")
ax.legend(fontsize=15)
fig.savefig("./figs/lyapunov x.svg", transparent=True, bbox_inches='tight')



fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.scatter(time, lyapunov_matrix_1[1], label="Energy: %0.5f" %energy_1)
ax.scatter(time, lyapunov_matrix_2[1], label="Energy: %0.5f" %energy_2)
ax.axhline(0, color="k", label="Separation of stable and chaotic behavior")

#ax.set_title("Lyapunov coefficient in y direction", fontsize=30)
ax.set_xscale("log")
ax.legend(fontsize=15)

fig.savefig("./figs/lyapunov x.svg", transparent=True, bbox_inches='tight')

### Lyapunov coefficients and energy

In [ ]:
energies = np.linspace(1/14, 1/4, 200)

lyapunov_matrix = []

for i in range(len(energies)):
    lyapunov_max = np.array([0,0])
    for j in range(10):
        initial_values = calc_initial_values(energies[i])

        _, solution_rk, _ = solve_RK(init=initial_values, steps=1000, check_energy=False)
        lyapunov_now = lyapunov_indexes(solution_rk)
        if lyapunov_max.max() < lyapunov_now.max():
            lyapunov_max = lyapunov_now
            
    lyapunov_matrix.append(lyapunov_max)
    
lyapunov_matrix = np.array(lyapunov_matrix).T

In [ ]:
x_interp = interp.interp1d(energies, lyapunov_matrix[0], kind="linear")
y_interp = interp.interp1d(energies, lyapunov_matrix[1], kind="linear")
energies_interp = np.linspace(energies.min(), energies.max(), 100000)
x_interp = x_interp(energies_interp)
y_interp = y_interp(energies_interp)

plt.figure(figsize=(10,10))

plt.plot(energies_interp, x_interp, label="x interpolation")
plt.plot(energies_interp, y_interp, label="y interpolation")

plt.scatter(energies, lyapunov_matrix[0], label="x data")
plt.scatter(energies, lyapunov_matrix[1], label="y data")

plt.axvline(1/12, color="k", linewidth=3, linestyle="-", label="E=1/12")
plt.axvline(1/9, color="k", linewidth=3, linestyle="--", label="E=1/9")
plt.axvline(1/7, color="k", linewidth=3, linestyle="-.", label="E=1/7")
plt.axvline(1/6, color="k", linewidth=3, linestyle=":", label="E=1/6")

plt.xlabel("Energies", fontsize=20)
plt.ylabel("Lyapunov coefficiens", fontsize=20)
#plt.yscale("log")

plt.legend(loc="lower right", fontsize=20)
plt.savefig("./figs/lyapunov.svg", transparent=True, bbox_inches='tight')
plt.show()

## Bifurcation diagram

In [ ]:
data = []
for energy in np.linspace(1/12, 1/6, 30):
    _, passing_velocity = poincare_section(energy, 10, calc_passing_velocity=True)
    n,bins = np.histogram(passing_velocity, bins=1000)
    data.append(n)
    
data = np.array(data).T

In [ ]:
plt.figure(figsize=(10,10))
plt.title('Bifurcation diagram of instantaneous velocity', fontsize=20)
plt.imshow(data, cmap="gray", aspect='auto', extent=[1/12,1/6,0,1])
plt.xlabel('Energy', fontsize=15)
plt.ylabel('possible velocities', fontsize=15)

For quantitative results I will use the Lyapunov coefficients for every dimension and try to adapt the
Shannon entropy.

## KAM theorem

In [ ]:
energy_1 = 1/12
energy_2 = 1/6
steps = 2000

initial_values = calc_initial_values(energy_1)
_, solution_rk, _ = solve_RK(initial_values, steps)
initial_values_2 = calc_initial_values(energy_2)
_, solution_2, _ = solve_RK(initial_values_2, steps)



plt.rcParams.update({'font.size': 12})
fig = plt.figure(figsize=(10,10))
#fig.suptitle('3D orbit with small perturbation', fontsize=30)

ax = fig.add_subplot(111, projection='3d')
ax.plot3D(solution_rk[0], solution_rk[1], solution_rk[3], color="k", linewidth=1)
#ax.set_title("Energy: %0.5f" %energy_1, fontsize=30)
ax.set_xlabel(r'$x$', fontsize=20)
ax.set_ylabel(r'$y$', fontsize=20)
ax.set_zlabel(r'$p_y$', fontsize=20)
plt.savefig("./figs/kam stable.svg", transparent=True, bbox_inches='tight')



plt.rcParams.update({'font.size': 12})
fig = plt.figure(figsize=(10,10))
#fig.suptitle('3D orbit with small perturbation', fontsize=30)

ax = fig.add_subplot(111, projection='3d')
ax.plot3D(solution_2[0], solution_2[1], solution_2[3], color="k", linewidth=1)
#ax.set_title("Energy: %0.5f" %energy_2, fontsize=30)
ax.set_xlabel(r'$x$', fontsize=20)
ax.set_ylabel(r'$y$', fontsize=20)
ax.set_zlabel(r'$p_y$', fontsize=20)
plt.savefig("./figs/kam unstable.svg", transparent=True, bbox_inches='tight')

plt.show()

## Relative area

In [ ]:
energies = np.linspace(1/14, 1/4, 50)
diverges_matrix = []
stable_matrix = []
for energy in energies:
    diverges = 0
    stable = 0
    for i in range(50):
        initial_values = calc_initial_values(energy)
        _, solution_rk, _ = solve_RK(init=initial_values, steps=1000, check_energy=False)
        if lyapunov_indexes(solution_rk).max() < 0:
            stable = stable + 1
    diverges_matrix.append(diverges)
    stable_matrix.append(stable)
    
stable_matrix = np.array(stable_matrix)
stable = stable_matrix / 50

In [ ]:
plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(10,10))

plt.scatter(energies, stable)
plt.plot(energies, stable)
plt.axvline(1/12, color="k", linewidth=3, linestyle="-", label="E=1/12")
plt.axvline(1/9, color="k", linewidth=3, linestyle="--", label="E=1/9")
plt.axvline(1/7, color="k", linewidth=3, linestyle="-.", label="E=1/7")
plt.axvline(1/6, color="k", linewidth=3, linestyle=":", label="E=1/6")

plt.xlabel("Energy", fontsize=20)
plt.ylabel("Relative area", fontsize=20)
plt.legend(fontsize=20)
plt.savefig("./figs/relarea.svg", transparent=True, bbox_inches='tight')